### Ant Colony Optimization

In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
def mod_relu(x):
    if x >1:
        return 1
    elif x < 0:
        return 0
    return x

In [4]:
class Node:
    def __init__(self, n_inputs, n_outputs):
        self.weights = [1 for i in range(n_outputs)]

In [5]:
def choose_path(weight_list):
    s = sum(weight_list)
    m = random.random()
    e = 0
    for i in range(len(weight_list)):
        e += weight_list[i]
        if m < e/s:
            return i

In [6]:
def ant_path(startint, goalint, network, network_length):
    g = []
    g.append(startint)
    for i in range(network_length-2):
        p = choose_path(network[i][g[-1]].weights)
        g.append(p)
    g.append(goalint)
    return g

In [7]:
def explore(ant_paths,input_val, pre_net_weight, network,network_length, expected_val):
    e = []
    o = []
    x = len(ant_paths)//2
    for i in range(0,2*x,2):
        a,b = ant_paths[i],ant_paths[i+1]
        n = input_val[i//2]
        n = mod_relu(n*pre_net_weight[i//2])
        m = n
        for j in range(len(a)-1):
            m = mod_relu(m*network[j][a[j]].weights[a[j+1]])
            n = mod_relu(n*network[j][b[j]].weights[b[j+1]])
        e.append(expected_val[0]-m)
        e.append(expected_val[1]-n)
    return e
        
        

In [8]:
def path_generator(input_layer,network,network_length):
    ant_paths = []
    for i in range(input_layer):
        ant_paths.append(ant_path(i,0,network,network_length))
    return ant_paths

In [9]:
e = explore(ant_paths,[0.8,0.363,0.9,0.76,0.19,0.56,0.49],pre_net_weight,network,5,[1,0])

NameError: name 'ant_paths' is not defined

In [10]:
def copynode(index,node,ant_paths):
    s = 0
    for i in ant_paths:
        if i[index] == node:
            s +=1
    return s

In [11]:
def backtrack(network,network_length,error,ant_paths, r = 1):
    s = len(ant_paths[0])
    for i in range(s):
        n = []
        m = []
        for j in range(len(ant_paths)):
            l = copynode(i,ant_paths[j][i],ant_paths)
            print(i,j,ant_paths[j][i])
            m.append(network[i][j//2].weights[ant_paths[j][i]])
            if l > 1 and ant_paths[j][i] in n:
                network[i][j//2].weights[ant_paths[j][i]] += error[j]/(l*3)
            else:
                network[i][j//2].weights[ant_paths[j][i]] =2*(network[i][j//2].weights[ant_paths[j][i]])/3 + error[j]/(l*3)
                if l > 1:
                    n.append(ant_paths[j][i])
        for k in range(len(error)):
            error[k] *= r*m[k]
    return network

In [13]:
data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')

In [14]:
from sklearn.preprocessing import MinMaxScaler
 
mx = MinMaxScaler()
cols = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education']
data[cols] = mx.fit_transform(data[cols])

In [15]:
data.drop(["ID","Mortgage","Personal Loan", "Securities Account","CD Account","Online","ZIP Code"],axis=1,inplace=True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         5000 non-null   float64
 1   Experience  5000 non-null   float64
 2   Income      5000 non-null   float64
 3   Family      5000 non-null   float64
 4   CCAvg       5000 non-null   float64
 5   Education   5000 non-null   float64
 6   CreditCard  5000 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 273.6 KB


In [17]:
x,y = data.iloc[:,:-1],data.iloc[:,-1]

In [18]:
from sklearn.model_selection import train_test_split

xtr,xt,ytr,yt = train_test_split(x,y,test_size = 0.3)

In [25]:
xtr = xtr.values
xt = xt.values
ytr = ytr.values
yt = yt.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [26]:
epoch = 11

In [27]:
pre_net_weight = [1 for i in range(5)]
network = [[Node(5,8) for i in range(7)],
          [Node(7,4) for i in range(8)],
          [Node(8,5) for i in range(4)],
          [Node(4,1) for i in range(5)],
          [Node(5,0) for i in range(1)]]

In [29]:
for i in range(epoch):
    for i in range(len(xtr)):
        paths = path_generator(5,network,5)
        e = explore(paths,xtr[i],pre_net_weight,network,5,ytr[i])
        backtrack(network,5,e,paths, r = 1)

IndexError: invalid index to scalar variable.